TODO
* Complete function documentation.
* Jail strategy is hard-coded as, in order, use a Get-Out-Of-Jail card, pay the fine, and roll the dice. In the future, strategy selection should be able to vary across players and be context-sensitive.
* Default method is simplistic. Need to code logic for what happens to defaulter's property.

### Import modules

In [10]:
import csv
import pandas as pd
import numpy as np
#import parameters as p

### Define functions

In [62]:
def get_players(n_players):
        
    assert n_players > 2, 'A game must have at least 2 players.'
    assert n_players < 8, 'A game must have no more than 8 players.'
    
    class Player(object):
        
        def __init__(self, player_id):
            self.id = player_id  # Identification number
            self.cash = 1500     # Cash on hand
            self.properties = [] # List of properties
            self.position = 0    # Board position
            self.jail_cards = 0  # Number of "Get Out Of Jail Free" cards
            self.jail_turns = 0  # Number of remaining turns in jail
            self.jail_strtg = '' # Jail strategy
        
        def move(self, roll, verbose=False):
            self.position += roll
            if self.position >= 40:
                self.position -= 40
                self.cash += 200
            if verbose:
                print 'Player {} to space: {}'.format(self.id, self.position)
        
        def buy(self, prop_id, price):
            self.properties.append(prop_id)
            self.cash -= price
        
        def pay(self, payee, payment):
            self.cash -= payment
            players[payee].cash += payment
        
        def go_to_jail(self):
            self.position = 10
            self.jail_turns = 3
        
        def choose_jail_strtg(self, rolled_double):
            if self.jail_cards > 0:
                self.jail_strtg = 'card'
                self.jail_turns = 0
                self.jail_cards -= 1
            elif self.cash >= 50:
                self.jail_strtg = 'pay'
                self.jail_turns = 0
                self.cash -= 50
            else:
                self.jail_strtg = 'roll'
                if rolled_double:
                    self.jail_turns = 0
                else:
                    self.jail_turns -= 1
                    if self.jail_turns == 0:
                        self.cash -= 50
                        
        def default(self):
            del players[self.id]
            
    return [Player(p) for p in range(n_players)]

In [60]:
def get_board(board_file):
        
    class Property(object):
        def __init__(self, name, position, price, rent):
            self.name = name
            self.position = position
            self.price = price
            self.price_mortgage = price / 2
            self.rent = rent
            self.rent_now = rent
            self.mortgage = False
            self.owner = 0

    class Street(Property):
        def __init__(self, name, position, color, price,
                     price_building, rent, rent_building):
                         Property.__init__(self, name, position, price, rent)
                         self.color = color
                         self.price_building = price_building
                         self.rent_monopoly = rent * 2
                         self.rent_building = rent_building
                         self.n_building = 0

    class Railroad(Property):
        def __init__(self, name, position, price, rent):
            Property.__init__(self, name, position, price, rent)
            self.rent_double = rent * 2
            self.rent_triple = self.rent_double * 2
            self.rent_monopoly = self.rent_triple * 2
    
    class Utility(Property):
        def __init__(self, name, position, price, rent):
            Property.__init__(self, name, position, price, rent)
            self.rent_monopoly = rent + 6
    
    class Tax(object):
        def __init__(self, price):
            self.price = price

    class Card(object):
        pass  
    
    class Chance(object):
        pass
    
    class Chest(object):
        pass
    
    class Jail(object):
        pass
    
    class Idle(object):
        pass
    
    board = []
    
    for _, r in pd.read_csv(board_file).iterrows():
        for case in switch(r['class']):
            if case('Street'):                
                board.append(Street(r['name'], r['position'], r['color'], r['price_buy'],
                                    r['price_build'], r['rent'], r['rent_build_1'], r['rent_build_2'],
                                    r['rent_build_3'], r['rent_build_4'], r['rent_build_5']))
            elif case('Railroad'):
                board.append(Railroad(r['name'], r['position'], r['price_buy'], r['rent']))
            elif case('Utility'):
                board.append(Utility(r['name'], r['position'], r['price_buy'], r['rent']))
            elif case('Tax'):
                board.append(Tax(r['price_buy']))
            elif case('Chance'):
                board.append(Chance())
            elif case('Chest'):
                board.append(Chest())
            elif case('Jail'):
                board.append(Jail())
            elif case('Idle'):
                board.append(Idle())
    
    return board

In [57]:
def roll_dice(check_double=True, verbose=False):

    roll = np.random.choice(np.arange(1, 7), 2)
    
    if verbose:
        print 'Die roll: {} and {}'.format(roll[0], roll[1])
    
    if check_double:
        return roll.sum(), roll[0] == roll[1]
    else:
        return roll.sum()

In [58]:
class switch(object):
    
    def __init__(self, value):
        self.value = value
        self.fall = False

    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration
    
    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args:
            self.fall = True
            return True
        else:
            return False

### Play the game

Get set

In [63]:
# Declarations
n_players = 5
board_file = 'board.csv'

# Get players and board (including properties)
players = get_players(n_players)
board = get_board(board_file)

# Start on round 1
game_round = 1

Go

In [ ]:
# Start game
while len(players) > 1:

    # Take turns
    for turn in range(n_players):

        # Double roll counter
        n_double_roll = 0

        # Continue turn until player rolls no doubles or goes to jail
        while True:

            # Roll dice
            roll, rolled_double = roll_dice(verbose=False)

            # Update double roll counter
            n_double_roll += (rolled_double).astype(int)

            # If player is in jail
            if players[turn].jail_turns > 0:

                # Select jail strategy
                players[turn].choose_jail_strtg(rolled_double)

                # If player is still in jail
                if players[turn].jail_turns > 0:
                    break

            # If player rolled less than 3 doubles
            if n_double_roll < 3:

                # Move player 
                players[turn].move(roll, verbose=False)

                # If no double rolled, end turn
                if not rolled_double:
                    break

            # Otherwise, send player to jail and end turn
            else:
                players[turn].go_to_jail()
                break

            # Now here is where we start interacting with the board
            type(board[4]).__name__
            http://code.activestate.com/recipes/410692/

    game_round += 1

    if game_round == 20:
        break

    # sum values across list
    # sum(players[i].cash for i in range(0,4))